In [ ]:
# create a map to display everything
from sepal_ui import mapping

Map = mapping.SepalMap()

In [ ]:
# create the model for testing purposes
from component import model
from datetime import datetime

alert_model = model.AlertModel()
alert_model.alert_type = "RECENT"
alert_model.alert_collection = "NRT"
alert_model.start = datetime.strptime("2022-01-01", "%Y-%m-%d")
alert_model.end = datetime.strptime("2022-12-31", "%Y-%m-%d")
alert_model.min_size = 0
alert_model.asset = "users/wiell/change-alert-cambodia"

In [ ]:
# create the aoi_model
from sepal_ui import aoi
from sepal_ui import color as sc
from sepal_ui import sepalwidgets as sw
import ee

ee.Initialize()

aoi_model = aoi.AoiModel(
    asset="users/bornToBeAlive/cambodia_alert_aoi", alert=sw.Alert()
)
Map.zoom_ee_object(aoi_model.feature_collection.geometry())
empty = ee.Image().byte()
outline = empty.paint(featureCollection=aoi_model.feature_collection, color=1, width=4)
Map.addLayer(outline, {"palette": sc.primary}, "aoi")

In [ ]:
# generate the grid
from component import scripts as cs
from ipyleaflet import GeoJSON
import json

x = 74
grid = cs.set_grid(aoi_model.gdf)
data = json.loads(grid[grid.index.isin([x])].to_json())
style = {"stroke": True, "color": "grey", "weight": 2, "opacity": 1, "fill": False}
ipygeojson = GeoJSON(data=data, style=style, name="grid")
Map.add_layer(ipygeojson)

len(grid)

In [ ]:
# display all alerts
a = alert_model
geom = ee.FeatureCollection(data)
alerts = cs.get_alerts(a.alert_collection, a.start, a.end, geom, a.asset)
Map.addLayer(
    alerts.select("alert").clip(geom),
    {"min": 1, "max": 2, "palette": ["red", "yellow"]},
    "alerts",
)
Map.zoom_ee_object(geom.geometry());

In [ ]:
# extract eh alerts
import geopandas as gpd

alert_clump = cs.get_alerts_clump(alerts=alerts, aoi=geom)
json = alert_clump.getInfo()
gdf = gpd.GeoDataFrame.from_features(json)
style = {"stroke": True, "color": "BLUE", "weight": 2, "opacity": 1, "fill": True}
ipygeojson = GeoJSON(data=json, style=style, name="alerts")
Map.add_layer(ipygeojson);

In [ ]:
Map

In [ ]:
from datetime import datetime, timedelta

start = datetime.strptime("2018-12-31", "%Y-%m-%d")
delta = timedelta(days=1183)

finish = start + delta

In [ ]:
type((finish - start).days)